In [4]:
import requests
import pandas as pd
import folium
import urllib
from pyproj import Proj, Transformer

콩나물 좌표계(다음)은 TM(중부원점) 좌표계의 2.5배수를 사용.  
즉 콩나물 좌표계에 0.4를 곱하고, TM -> WGS64를 사용하면 위경도를 얻을 수 있음  
중부원점 좌표계는 epsg:5181을 사용하면 된다.  
(자바스크립트와 비교하면 오차범위 내에 존재하는 것 같다)

In [5]:
# epsg:5181 - 콩나물 좌표계(카카오맵) / 2.5
# epsg:4326 - WGS84 경위도
transformer = Transformer.from_crs(5181, 4326)

In [11]:
Proj(4326).to_proj4()

'+proj=longlat +datum=WGS84 +no_defs'

In [7]:
def get_tile_information(query, x, y, zoom=7):
  baseurl = 'https://tilesrch.map.daum.net/tmap/msearch/L{zoom}/{x}/{y}.png?output=jsonp&query={query}&dimg=ns'
  # baseurl = 'http://localhost:5094/tmap/msearch/L{zoom}/{x}/{y}.png?output=jsonp&query={query}&dimg=ns'
  res = requests.get(
    baseurl.format(query=urllib.parse.quote(query), x=x, y=y, zoom=zoom),
    headers={
      'Referer': 'https://map.kakao.com/',
      'User-Agent': 'curl/7.64.1',
    },
  )
  return [{
    **data,
    **(lambda wgs: {
      'lat': wgs[0],
      'long': wgs[1],
    })(transformer.transform(data['y'] * 0.4, data['x'] * 0.4))
  } for data in res.json()]

In [116]:
xrange = range(109, 147 + 1)
yrange = range(41, 84 + 1)
data = [
  inner
    for data in
      [
        get_tile_information('도서관', xtile, ytile)
          for ytile in yrange
            for xtile in xrange
      ]
      for inner in data
]

In [8]:
get_tile_information('도서관', 63, 28, 8)

[{'x': 514340,
  'y': 1155958,
  'ID': '8251560',
  'NAME': '노원정보도서관',
  'CATE': '도서관',
  'lat': 37.66107145835328,
  'long': 127.0650080994179},
 {'x': 509708,
  'y': 1151371,
  'ID': '12449826',
  'NAME': '도봉문화정보도서관',
  'CATE': '도서관',
  'lat': 37.644549935251156,
  'long': 127.04399991930676},
 {'x': 507948,
  'y': 1145918,
  'ID': '23885659',
  'NAME': '강북문화정보도서관',
  'CATE': '도서관',
  'lat': 37.62490034849652,
  'long': 127.03601352346533},
 {'x': 514943,
  'y': 1149928,
  'ID': '8224425',
  'NAME': '서울특별시교육청 노원평생학습관',
  'CATE': '도서관',
  'lat': 37.63933822199714,
  'long': 127.06772196541792},
 {'x': 502868,
  'y': 1153659,
  'ID': '15992077',
  'NAME': '도봉도서관',
  'CATE': '도서관',
  'lat': 37.65280322376875,
  'long': 127.01300017886993},
 {'x': 518283,
  'y': 1151563,
  'ID': '863369993',
  'NAME': '불암도서관',
  'CATE': '도서관',
  'lat': 37.6452209938699,
  'long': 127.08286544190936},
 {'x': 506125,
  'y': 1156230,
  'ID': '12120008',
  'NAME': '학마을도서관',
  'CATE': '도서관',
  'lat': 37.66206

In [117]:
data_df = pd.DataFrame(data)
data_df

x        y          ID       NAME   CATE        lat        long
0     361928  1096749  2114286628    용유중앙도서관  작은도서관  37.446053  126.375872
1     360585  1101670   624204375   하늘꿈작은도서관  작은도서관  37.463755  126.369652
2     368650  1095748   882703750   용유동작은도서관  작은도서관  37.442602  126.406284
3     372019  1148187    21812755    화도작은도서관  작은도서관  37.631658  126.420050
4     374244  1147610  2127734151    초암작은도서관  작은도서관  37.629628  126.430148
...      ...      ...         ...        ...    ...        ...         ...
5364  789155  1236957    33707126  다독다독작은도서관  작은도서관  37.945638  128.315849
5365  786948  1241805    33752490  우리동네작은도서관  작은도서관  37.963216  128.306117
5366  789809  1239578    18606415      기린도서관    도서관  37.955048  128.318993
5367  794622  1306225    21814145   백담솔방울도서관  작은도서관  38.194908  128.345284
5368  793990  1306649  2098355749   백담솔방울도서관  작은도서관  38.196468  128.342427

[5369 rows x 7 columns]

In [29]:
data_df[['long', 'lat']].mean().values.tolist()

[135.176492298524, 37.207174882423985]

In [118]:
m = folium.Map(data_df[['lat', 'long']].mean().values.tolist())
popup = folium.Popup(min_width=200, max_width=200)
for index in data_df.index:
  folium.Marker(
    location=[data_df.loc[index, 'lat'], data_df.loc[index, 'long']],
    fill=True,
    popup=folium.Popup(data_df.loc[index, 'NAME'], max_width=400),
    ).add_to(m)
m.fit_bounds([
  data_df[['lat', 'long']].min().values.tolist(),
  data_df[['lat', 'long']].max().values.tolist(),
]) 
m